# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

We are trying to estimate a target function that is able to distinguish between two discerete classes of students : the passed students and the failed students. these two classes are discrete and countable therefore it is a classificaiton problem . 
if it is was a regression problem we would have tried to estimate a target function that is able to predict the grades of the students given features vectors.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [17]:
# Import libraries
import numpy as np
import pandas as pd
import random as rand
#from sklearn import cross_validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn import cross_validation
from sklearn.metrics import classification_report

In [18]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print ("Student data read successfully!")
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [19]:

n_students = student_data.shape[0]
# removing the column labels passed from the total count
n_features = student_data.shape[1] - 1
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]
n_failed = student_data[student_data['passed'] == 'no'].shape[0]
grad_rate = float(n_passed)/ n_students * 100
print ("Total number of students: ",n_students)
print ("Number of students who passed:",n_passed)
print ("Number of students who failed: ",n_failed)
print ("Number of features: ",n_features)
print ("Graduation rate of the class",format(grad_rate))

Total number of students:  395
Number of students who passed: 265
Number of students who failed:  130
Number of features:  30
Graduation rate of the class 67.08860759493672


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [20]:
# Extract feature (X) and target (y) columns
# mind the columns attribute 
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print ("Feature column(s):-\n{}",format(feature_cols))
print ("Target column: {}",format(target_col))

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print ("\nFeature values:-")
print( X_all.head() ) # print the first 5 rows

Feature column(s):-
{} ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: {} passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   .

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [21]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print ("Processed feature columns ({}):-\n{}",(len(X_all.columns),str(X_all.columns)))

Processed feature columns ({}):-
{} (48, "Index(['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R',\n       'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T',\n       'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other',\n       'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health',\n       'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course',\n       'reason_home', 'reason_other', 'reason_reputation', 'guardian_father',\n       'guardian_mother', 'guardian_other', 'traveltime', 'studytime',\n       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',\n       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',\n       'Walc', 'health', 'absences'],\n      dtype='object')")


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [22]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
# As noted by the reviewer
# We will use stratify flag on, to assure proper sampling from each population.
# Also the random state of the samplers is noted in case results need to be replicated.
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify=y_all, test_size=0.24, random_state=50)

print ("Grad rate of the training set: {:.2f}%".format(100 * (y_train == 'yes').mean()))
print ("Grad rate of the testing set: {:.2f}%".format(100 * (y_test == 'yes').mean()))

Grad rate of the training set: 67.00%
Grad rate of the testing set: 67.37%


## 4. Training and Evaluating Models
Choose three supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

### A. Decision Trees (DT)
- What are the general applications of this model? What are its strengths and weaknesses?

 - Strength 
 
Decision trees can model inductive inference, and it can approximate discrete value functions, which applies to our case . it is robust to noisy data and can represent discrete functions. The noise source of the data, in this instance, can be the data entry filled by the students or the administration. 

 - Weaknesses 
  
The main concern for using the decision trees is its potential to overfit the data if no post rule pruning is applied. Sklearn implementation includes regression and classification. in addition to posting pruning rules to control the overfitting.

 - Applications of this model
 
 Researchers reported the use of Decision trees in predicting indivdual use of library books, classifying stars and cosmic rays from the image of bubble telecscope .

http://booksite.elsevier.com/9780124438804/leondes_expert_vol1_ch3.pdf


- Given what you know about the data so far, why did you choose this model to apply?

The data in the excel sheet are discrete data that represents different classes. An attribute like ages can be still considered as discrete data for using the decision tree.

- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on the training set and F<sub>1</sub> score on the test set, for each training set size.



### B. KNN

 - What are the general applications of this model? What are its strengths and weaknesses?
 
 - Strengths
 
K nearest neighbors can be used as a classifier and a regressor. It belongs to the family of lazy learning algorithms. Also, it can be used in unsupervised learning. 
K-NN is very simple to implement and is highly effective for many problems. It is robust to noise and when supplied with enough data is quite effective.

  - Weaknesses 
The weakness of k nearest neighbors is that it requires a lot of data for it to be efficient therefore it can consume a lot of memory. K nearest neighbors inference model is postponed to the prediction phase at that stage it will start querying the instances to predict the labels. Therefore it needs a memory efficient implementation, and it is slower in its prediction compared to other learning algorithms.

  - Applications of the model
  
 It is used in satellite imagery and handwritten digits recognition.

- Given what you know about the data so far, why did you choose this model to apply?

The data can be considered a mixed data between continuous and discrete therefore K-NN can be implemented. Also, this type of demographic information can be very suited for K-NN since similar labels can have neighboring feature. Therefore, a close neighbor might be enough to give optimal information on the instance we are trying to predict.


### C. Naive Bayes

 - What are the general applications of this model? What are its strengths and weaknesses?

Bayes based classifier have become popular classifier as their performance sometimes matches other classifiers. Naiive base classifier are built on the assumption that the features of an instances are independent therefore it simplfies the search space and make the computation tractable .
 
 - Strengths
 
Naiive bayes are easy to implement 

  - Weaknesses 
  
Naive bayes weakness is that it assumes that the attributes/features of a given instances are indepedent, though this assumption works very well for classification, it is usually not the optimal classification and it provides a less than optimal probability predictive model.


  - Applications of the model
  
Naiive bayes is reported to be used in text classification ( suggesting articles for readers)and researchers are using NB augmented with decision trees in decision based medicine. 

- Given what you know about the data so far, why did you choose this model to apply?

I think , the assumption that the feautres are independent in this data set is safe therefore NB can be applied. 


### D. RBF SVM

- What are the general applications of this model? What are its strengths and weaknesses?

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression, and outlier detection, the SVM implementation is done in the dynamic programming language.

 - The strengths of SVM
 
SVM can be used for models that have a large feature space Also it can be sufficient for the cases where the number of features is larger than the number of the samples. SVM Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient. 
 
 - Weakness of SVM
 
SVM has shown weak performance once the number of features is much larger than the number of the samples.  SVMs are not directly capable of producing probability estimates; these are calculated using an expensive five-fold cross-validation.

 - General Applications of SVM
 
 SVM is reported to be used in Image Clustering, Cancer Diagnosis and Protein Secondary Structure Prediction and Ocean Color Remote Sensing. 
 http://www3.ntu.edu.sg/home/elpwang/pdf_web/05_svm_basic.pdf


- Given what you know about the data so far, why did you choose this model to apply?

The number of the features is smaller than then number of samples. SVM can handle some 30 features.

### E. AdaBoost

 

- What are the general applications of this model? What are its strengths and weaknesses?
 
Ada-boost algorithm belongs to boosting family of algorithm which is also considered also as part of ensemble learning algorithm. The Ada-boost is an iterative algorithm which assumes a weak learner as a base learner( doing slightly better thant chance) , at each iteration the algorithm assigns higher weights to miss classified labels .

- The strengths of ADA-Boost
Ada-Boost algorithm can work with very heterogeneous data, and their performance matches other classifiers, 

- The weakness of ADA-boost

In the practical application of the Ada boost , the algorith perforance is influenced by uniform noise.

the weakness of boosting algorithm is that they don't scale with the size of the data cause of their sequential nature. 
 
If the classifier is too complex the algorithm might overfitt the data. 

 - Applications of ADA-Boost

Viola and Jones used ADA-boost to detect faces with rectangles as weak classifiers.


- Given what you know about the data so far, why did you choose this model to apply?
Though most of the data is considered discrete, some of the data can be regarded as continuous as well. Therefore using Adaboost is proper in this situation.


In [33]:
# Train a model

# Review comments : 



#1) It's fine to try out some parameter tuning in this section, but make sure you're also following the implementation guidelines and producing results for the default params of each model (this will enhance our comparison to the later grid search tuning).

2#) Also, in order to make your results reproducible by others, you should specify random states for any classifiers as needed.


classifiers = [
    # Decision tree splitter is set by default to best i have set it explictely .
    DecisionTreeClassifier(),
    
    # Knn has no random state to specify however it has algorithm automatic selection. it should not affect the performance 
    # of knn  in term f1_score but it might affect the computation time so i will leave it on auto . wjich selects the best 
    # computation efficient implementation .
    
    KNeighborsClassifier(),
    GaussianNB(),
    # gamma affects the influence of training points  on the support vector, a low gamma it means it includes less points in the training 
    # a high gamma more points are included in the training , a high C means a more complex decision surface (prone to overfitting)
    # a low C mmore smooth decision surface, which means more missclassified points(high bias)
    SVC(gamma=2, C=1),
    AdaBoostClassifier(),
    ]

pd.options.display.float_format = '{:,.2f}'.format

df_100 = pd.DataFrame(0.,index=['DT','KNN','NB','SVM RBF','Adaboosting'],columns=['training_time(ms)', 'cross_validation_training_score',
                                                                               'f1_score_training','f1_score_testing','prediction_time'])
df_200 = pd.DataFrame(0.,index=['DT','KNN','NB','SVM RBF','Adaboosting'],columns=['training_time(ms)', 'cross_validation_training_score',
                                                                               'f1_score_training','f1_score_testing','prediction_time'])
df_300 = pd.DataFrame(0.,index=['DT','KNN','NB','SVM RBF','Adaboosting'],columns=['training_time(ms)', 'cross_validation_training_score',
                                                                               'f1_score_training','f1_score_testing','prediction_time'])




In [34]:
# functions that trains the classifiers , predicts the lables and 
# calculate the amount spent during training and testing 

from sklearn.metrics import f1_score
import time

# Train classifier and caluclate time spent in training
def train_classifier(clf, X_train, y_train):
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    time_Spent = (end - start)*1000;
    return time_Spent
# Predict the labels , using training and testing data
def predict_labels(clf, features, target):
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    time_Spent = (end - start)*1000;
    return f1_score(target.values, y_pred, pos_label='yes'), time_Spent

# call the previous functions train_classifier and predict_labels. 

def train_predict(clf, X_train, y_train, X_test, y_test):
    #print ("------------------------------------------")
    #print ("Training set size: {}".format(len(X_train)))
    training_time = train_classifier(clf, X_train, y_train)
    train_f1_score, training_prediction_time = predict_labels(clf, X_train, y_train)
    #print ("F1 score for training set: %0.3f" % (train_f1_score))
    #print ("Predicting labels using %s" % (clf.__class__.__name__))
    test_f1_score , test_prediction_time =(predict_labels(clf, X_test, y_test))
    #print( "Prediction time (msecs): %0.5f" % (test_prediction_time))
    #print ("F1 score for testing set: %0.3f" % (test_f1_score)) 
    return  training_time, train_f1_score, training_prediction_time, test_f1_score , test_prediction_time



In [38]:
# Cross validation settings 
FOLDS = 3;
TYPE_OF_SCORING ='f1_weighted';
# TODO: Choose a model, import it and instantiate an object
kk = 0

# switch between classifiers , classifiers2, classifiers3
for clf in classifiers:

# Fit model to training data
 #print ("Training %s" % (clf.__class__.__name__))
 
 scores = cross_validation.cross_val_score(clf, X_train, y_train,cv=FOLDS, scoring=TYPE_OF_SCORING) 
 mean_scores = scores.mean();

 
 df_100['cross_validation_training_score'][kk] =  mean_scores;
 df_200['cross_validation_training_score'][kk] =  mean_scores;
 df_300['cross_validation_training_score'][kk] =  mean_scores;
# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
 array_of_training_slice = [100 , 200, 300]
 X_train_reduced = X_train[:array_of_training_slice[0]]
 y_train_reduced = y_train[:array_of_training_slice[0]]
 training_Time,tr_f1_score,tr_prediction_time, test_f1_score,test_prediction_time= train_predict(clf,X_train_reduced,
  
                                                                                                 y_train_reduced,X_test,y_test)
 df_100['training_time(ms)'][kk]=  training_Time
 df_100['f1_score_training'][kk] = tr_f1_score;
 df_100['f1_score_testing'][kk] = test_f1_score
 df_100['prediction_time'][kk] = test_prediction_time
    
    
 X_train_reduced = X_train[:array_of_training_slice[1]]
 y_train_reduced = y_train[:array_of_training_slice[1]]
 training_Time,tr_f1_score,tr_prediction_time, test_f1_score,test_prediction_time= train_predict(clf,X_train_reduced,
             
                                                                                                 y_train_reduced,X_test,y_test)
 df_200['training_time(ms)'][kk]=  training_Time
 df_200['f1_score_training'][kk] = tr_f1_score;
 df_200['f1_score_testing'][kk] = test_f1_score
 df_200['prediction_time'][kk] = test_prediction_time
    
    
    
 X_train_reduced = X_train[:array_of_training_slice[2]]
 y_train_reduced = y_train[:array_of_training_slice[2]]
 training_Time,tr_f1_score,tr_prediction_time, test_f1_score,test_prediction_time= train_predict(clf,X_train_reduced,
                                                                                                 y_train_reduced,X_test,y_test)
 df_300['training_time(ms)'][kk]=  training_Time
 df_300['f1_score_training'][kk] = tr_f1_score;
 df_300['f1_score_testing'][kk] = test_f1_score
 df_300['prediction_time'][kk] = test_prediction_time
 kk= kk +1;
print('------------------------100 Training samples ----------------------------')
print(df_100)
print('------------------------200 Training samples ----------------------------')
print(df_200)
print('------------------------300 Training samples ----------------------------')
print(df_300)


/home/nadim/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nadim/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nadim/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


------------------------100 Training samples ----------------------------
             training_time(ms)  cross_validation_training_score  \
DT                        0.88                             0.65   
KNN                       0.63                             0.62   
NB                        0.98                             0.64   
SVM RBF                   2.14                             0.54   
Adaboosting              62.14                             0.65   

             f1_score_training  f1_score_testing  prediction_time  
DT                        0.89              0.68             0.22  
KNN                       0.87              0.79             1.74  
NB                        0.78              0.74             0.37  
SVM RBF                   1.00              0.81             1.15  
Adaboosting               0.98              0.76             6.73  
------------------------200 Training samples ----------------------------
             training_time(ms)  cross_val

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is the most appropriate based on the available data, limited resources, cost, and performance?

I chose the K-NN as the best model for this data. K-NN has shown high performance throughout the sample training size of 100, 200,300 of f1_score_testing of 0.79, 0.73 and 0.74. SVM-RBF shows higher f1-score for small sample size however it is training time and computing efficiency is almost doubling the size of the data and its prediction time is similar to K-NN. At 300 sample size, K-NN performance is as good as SVM-RBF with less prediction time and training time.
Adaptive boosting performed better than K-NN for all training sample size, slightly outperforming K-NN for a training sample size of 300. However, the training_time is much bigger than the training time for K-NN and also the prediction time is twice as the K-NN.
Given the limited resources cost and performance I would say the K-NN is the best model. Please note that Adaptive boosting training time is almost 50 ms, on an Intel i7 four cores. I consider this setup as a standard computing  configuration. Depending on the available hardware of the school and the available data , I would say Ada-boosting is the best choice, however if the number of data will increase then the best choice so far will be K-NN .

- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model was chosen is supposed to work (for example, if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).

K-NN algorithm stands for K nearest neighbors, where K is the number of neighbors that are taken into consideration while classifying a student. Neighbors have close properties to the data we are trying to predict. For example let us assume we have five students with absences: 10, 12,20, 2,5  and the first three failed an exam while the last two passed. A student with 15 absences will most likely fail an exam, and a student of 4 absences will most likely pass an exam. The student with 15 absences will most likely fail the exam because for K = 3, or the closest three neighbors have 10, 12 and 20 absences, and they failed the exam. On the opposite end of the spectrum, the student with four absences might pass the exam because for K =2, or the closest two neighbors passed the exam.
Now for the case of a student with six absences what K-NN might predict?  It will depend on the number of neighbors we will take into consideration. If we take k = 3, the nearest neighbors are  students with 10 absences, 5 and 2 absences where one has failed, and the other two have passed respectively; the student will most likely pass the exam, however, if we take into consideration all five neighbors k =5, the student will most likely fail.



- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least three settings. Use the entire training set for this.

One of the important parameter to tune in K-NN is the number of neighbors.also if we need to weight the distances from the neighbors or not. 
- What is the model's final F<sub>1</sub> score?
0.79, which is better than for the default setting neighbors =5; 

In [36]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.grid_search import GridSearchCV 
import sklearn.metrics as metrics

number_Neighbours_knn = {'n_neighbors' : np.arange(1,100)}
clf = KNeighborsClassifier();



#f1_scorer = metrics.make_scorer(f1_score,pos_label=None,average='weighted');

f1_scorer = metrics.make_scorer(f1_score,pos_label='yes')
classifier= GridSearchCV(clf,number_Neighbours_knn,f1_scorer)
classifier.fit(X_train,y_train)

y_predict = classifier.predict(X_test)



print ("Best model parameter: " + str(classifier.best_params_))
print('F1_score',f1_score(y_test.values, y_predict, pos_label='yes'))
print (classification_report(y_test, classifier.predict(X_test)))

Best model parameter: {'n_neighbors': 34}
F1_score 0.794871794872
             precision    recall  f1-score   support

         no       0.33      0.03      0.06        31
        yes       0.67      0.97      0.79        64

avg / total       0.56      0.66      0.55        95

